## Assignment one. 
  
Nawen Gong  

Kaggle name: Noven_G

In [ ]:
import gzip
from collections import *
import numpy as np
from random import sample, randint
from sklearn.metrics import accuracy_score
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import os.path
import pprint
import random
from operator import itemgetter

In [712]:
def readGz(f):
  for l in gzip.open(f):
    yield eval(l)

data = list(readGz('train.json.gz'))

### Shuffling data as we may not be able to run on the whole dataset

In [1700]:
random.shuffle(data)
#data[:-50]

In [1701]:
train = data[0:100000]

val = data[100000:]

### Flatten the category and get the count of each category

In [1703]:
def flatten(items, seqtypes=(list, tuple)):
    for i, x in enumerate(items):
        while i < len(items) and isinstance(items[i], seqtypes):
            items[i:i+1] = items[i]
    return items

In [1704]:
category = [d['categories'] for d in data]

In [1705]:
fcat = flatten(category[:])
cat = list(set(fcat))

In [1706]:
hist = defaultdict(int)
for i in fcat:
    hist[i] += 1

In [1721]:
sorted_hist = sorted(hist.items(), reverse=True, key=operator.itemgetter(1))

In [ ]:
### get the count of each itemID

In [1707]:
hist_bis = defaultdict(int)
for i in data:
    hist_bis[i['itemID']] += 1

In [1708]:
sorted_hist_bis = sorted(hist_bis.items(), reverse=True, key=operator.itemgetter(1))

### get the most popular 10%, 20% 50% 80% and 90% items

In [1711]:
total_items = 0

for k,v in sorted_hist_bis:
    total_items += v

def popular_sorter(cutoff):
    print(total_items)
    popular_items = set()
    count = 0
    for k,v in sorted_hist_bis:
        popular_items.add(k)
        count += v
        if (count >= cutoff * total_items):
            break
    return popular_items.copy()

In [1712]:
popular_items_10 = popular_sorter(0.1)
popular_items_20 = popular_sorter(0.2)
popular_items_50 = popular_sorter(0.5)
popular_items_80 = popular_sorter(0.8)
popular_items_90 = popular_sorter(0.9)

200000
200000
200000
200000
200000


### Get a validation set with half negative samples

In [1713]:
unique_reviewer1 = list(set([d['reviewerID'] for d in data]))
unique_business1 = list(set([d['itemID'] for d in data]))

In [1715]:
visited_pairs = []
for datum in data:
    visited_pairs.append((datum['reviewerID'], datum['itemID']))
visited_pairs = set(visited_pairs)
len(visited_pairs)

200000

In [1716]:
non_visted_pairs = set()
counter = 100000
while counter:
    random_item_id = random.sample(unique_business1, 1)[0]
    random_userID = random.sample(unique_reviewer1, 1)[0]
    if ((random_userID, random_item_id) not in visited_pairs
        and (random_userID, random_item_id) not in non_visted_pairs):
            non_visted_pairs.add((random_userID, random_item_id))
            counter -= 1
            

In [1717]:
sampled_dataset = [[d[0], d[1]] for d in non_visted_pairs]
y_sampled = [0]*len(sampled_dataset)

In [1718]:
x_train1 = [[d['reviewerID'], d['itemID']] for d in train]
y_train1 = [1]*len(x_train)

In [1719]:
val_data1 = [[d['reviewerID'], d['itemID']] for d in val]
y_validation1 = [1]*len(val_data1)

In [1720]:
val_data1 = val_data1 + sampled_dataset

x_val1 = val_data1
y_val1 = y_validation1 + y_sampled

In [1721]:
# sorted_hist = sorted(hist.items(), reverse=True, key=operator.itemgetter(1))

### Basis for creating category list to use in the model

* Helpful (ideas):
    + Opposites:
        ('Women', 198548),
        ('Men', 80021),
        
        ('Petite', 20100),
        ('Big & Tall', 12217),
        ('Plus-Size', 11704),
        
        ('Girls', 3046),
        ('Boys', 2509),
        
        ('Baby Girls', 2120),
        ('Baby Boys', 2069),
   
    + Broad categories:
        ('Intimates', 28040),
        ('Jewelry', 23468),
        ('Sports & Outdoors', 17663)
        ('Novelty', 28807)
        ('Fashion', 6629)
        ('Shoes', 61746)
        

In [1723]:
catCuratedList = [
    'Women',
    'Men',
    'Petite',
    'Big & Tall',
    'Plus-Size',
    'Girls',
    'Boys',
    'Baby Girls',
    'Baby Boys',
    'Intimates',
    'Jewelry',
    'Sports & Outdoors',
    'Novelty',
    'Fashion',
    'Shoes',
    'Bras',
    'Accessories',
    'Jeans',
    'Tops & Tees',
    'Socks',
    'Underwear',
    'Baby'
]


In [ ]:
catCuratedDict = dict(zip(catCuratedList, range(len(catCuratedList))))

In [1724]:
unique_reviewer = list(set([d['reviewerID'] for d in train]))
unique_business = list(set([d['itemID'] for d in train]))
unique_category = [d for d in catCuratedDict] #[d[0] for d in sorted_hist[:10]]

In [1726]:
reviewerIds = dict(zip(unique_reviewer, range(len(unique_reviewer))))
businessIds = dict(zip(unique_business, range(len(unique_business))))
catIds = catCuratedDict
num_users = len(unique_reviewer)
num_business = len(unique_business)

### Creating a user/category matrix and user/category weighted matrix

In [1728]:
def getCatIds(datum, ref_cats):
    cats = []
    flat = flatten(datum['categories'])
    for f in flat:
        if f in ref_cats:
            cats.append(catIds[f])
#    print(cats)
    return cats

In [1729]:
categories = np.zeros((num_users, len(unique_category)))
for d in train:
    uId = reviewerIds[d['reviewerID']]
    cats = getCatIds(d, unique_category)
    for c in cats:
        categories[uId][c] = 1

In [1731]:
weighted_categories = np.zeros((num_users, len(unique_category)))
weighted_categories.fill(0)
for d in train:
    uId = reviewerIds[d['reviewerID']]
    cats = getCatIds(d, unique_category)
    for c in cats:
        weighted_categories[uId][c] += 1

### Creating an item/category matrix and item/category weighted matrix

In [1733]:
catToitem = np.zeros((num_business, len(unique_category)))
for d in train:
    bId = businessIds[d['itemID']]
    cats = getCatIds(d, unique_category)
    for c in cats:
        catToitem[bId][c] = 1

In [1734]:
weighted_catToitem = np.zeros((num_business, len(unique_category)))
for d in train:
    bId = businessIds[d['itemID']]
    cats = getCatIds(d, unique_category)
    for c in cats:
        weighted_catToitem[bId][c] += 1

In [1791]:
def predict_purchase_baseline(uid, bid):
    
    if bid not in unique_business:
        return 0

    if bid in popular_items_10:
        return 1
    elif uid not in unique_reviewer:
        if bid in popular_items_50:
            return 1
        return 0
    else:
        b = businessIds[bid]
        u = reviewerIds[uid]
        bc = np.atleast_2d(weighted_catToitem[b])
        bc_jac = catToitem[b]
        bc_mag = np.linalg.norm(np.array(bc_jac))
       # print(bc)
        #print(bc_mag)
        uc = np.atleast_2d(weighted_categories[u])
        uc_jac = categories[u]
        uc_mag = np.linalg.norm(np.array(uc_jac))
        #print(uc)
        #print(uc_mag)

        cos = cosine_similarity(bc, uc)
        jac = jaccard_similarity_score(bc_jac, uc_jac, normalize=True)

       # cos = np.linalg.norm()
       # print(cos)

        if bid in popular_items_20 and cos > 0:
            return 1
        elif cos >= 0.40 and bid in popular_items_50:

        
           # print(bc)
           # print(uc)
           # print(cos)
        
            return 1
        elif bid in popular_items_80 and cos >= 0.60:
            return 1 
        elif cos >= 0.80:
            #if uc_mag < 2 or bc_mag < 2:
               # print(bc)
             #   print(uc)
               # print(cos)
               # return 0
            return 1

      #      else:
       #         print(cos)
        #        print(jac)
       #         for d in train:
        #            if bid == d['itemID']:
         #               print("ITERM: " + str(d))
          #          if uid == d['reviewerID']:
           #             print("USER: " + str(d))

    
    #print(bc)
   # print(uc)
   # print(cos)
    return 0



#### Test on validation set, due to computing time, only test 20000 data.

In [1793]:
test_range = slice(90000,110000)

In [1794]:
pred_y=[]
for d in x_val1[test_range]:
    
    uid = d[0]
   # print(uid)
    bid = d[1]
   # print(bid)
    pred_y.append(predict_purchase_baseline(uid,bid))

In [1795]:
accuracy_score(pred_y, y_val1[test_range])

0.6841

In [1796]:
predictions = open("predictions_Purchase.txt", 'w')
for l in open("pairs_Purchase.txt"):
  if l.startswith("reviewerID"):
    #header

    predictions.write(l)
    continue
  uid,bid = l.strip().split('-')
  predictions.write(uid + "-" + bid + "," + str(predict_purchase_baseline(uid, bid)) + "\n")
predictions.close()